Load Libraries

In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta
import plotly.io as pio

Initialize Tensorflow

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

DB Config

In [ ]:
#####DB Config
host= "DevEdgeV32"
port=27017
all_data= True
get_preprocessed_data= False

Set parameters

In [ ]:
preprocessor_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/preprocessor/'
preprocessor_name='preprocessor.p'
model_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/model/'
model_name= 'model.h5'
model_input_Columns=  ['t_bett', 't_spindle','t_motor', 'M8','M121', 'M127', 'M7']#['t_bett','t_motor', 't_spindle','M8', 'M121', 'M127', 'M7']
output_variable= 'welle_z'
prediction_variable= 'prediction'
time_shift= +1
window=60
response_time= 30
time_relevance= 30
recording_time= 5

Remove t_bett function

In [ ]:
def remove_t_bett(data: np.ndarray):
    result= data[:, 1:]
    result[:,0]= result[:,0] - data[:,0]
    result[:,1]= result[:,1] - data[:,0]
    return result

In [ ]:
def rounding(signals: np.ndarray):
    ##rounding t_motor to nearest integer
    signals[:,1]= np.round(signals[:,1])
    signals= np.round(signals, 2)
    return signals

def rounding2(signals: np.ndarray):
    return np.round(signals, 5)


Load Model and Preprocessor

In [ ]:
### load models
pipeline= load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
model= load_model(str(model_dir + model_name), compile= False)
print('Model is loaded successfully')
print (model.input)

Set time priod needed

In [ ]:
#start= datetime.strptime("2022-09-20T22:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")# Trocken iso
#end= datetime.strptime ("2022-09-21T08:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
# start= datetime.strptime("2023-01-20T10:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") # Trocken Fräsen
# end= datetime.strptime ("2023-01-20T23:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
# start= datetime.strptime("2023-01-23T10:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") # nass Fräsen
# end= datetime.strptime ("2023-01-23T17:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
#start= datetime.strptime("2023-01-31T17:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
#end= datetime.strptime ("2023-01-31T23:59:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
start= datetime.strptime("2023-02-27T00:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime ("2023-03-03T00:59:59.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")

Get Data from database

In [ ]:
## Read raw and preprocessing data
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db.modelLogs.find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
last_prediction= -500
### Go through events in DB
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
    ##### Get only data points to be predicted
        if not all_data:
            if 'given2model' in record:
                if not record['given2model']:
                    continue
            elif record[prediction_variable] == last_prediction:
                continue
            else:
                last_prediction= record[prediction_variable]
        #####
        if 'given2model' in record:
           record['raw_data']['given2model'] =record['given2model']
        keys= record['raw_data'].keys()
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
        #print ('before',record)
        record['raw_data']['date']=[record['date'] + timedelta(hours= time_shift)]
        record['raw_data'][output_variable]=[record[output_variable]]
        record['raw_data'][prediction_variable]=[record[prediction_variable]]
        record['raw_data']['DRZ2']= float(record['DRZ2'])
        #record['raw_data']['DRZ2'][0]= float(record['raw_data']['DRZ2'][0])
        record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0], decimals=2)
        record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]), decimals= 2)
        #####Add preprocessed Data##########
        if get_preprocessed_data:
            record['raw_data']['0']= record['preprocessed_data'][0]
            record['raw_data']['1']= record['preprocessed_data'][1]
            record['raw_data']['2']= record['preprocessed_data'][2]
            record['raw_data']['3']= record['preprocessed_data'][3]
            record['raw_data']['4']= record['preprocessed_data'][4]
            record['raw_data']['5']= record['preprocessed_data'][5]
        ####################################
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
signals['t_motor']= signals['t_motor'].apply(np.float32)
# signals['db_prediction_abs_error']= signals[prediction_variable] -signals[output_variable]
# signals['db_prediction_abs_error']= 1000 * signals['db_prediction_abs_error']
signals.reset_index(inplace= True)
####Apply enterpolation###
upsampled_variable= output_variable#str('interpolated_'+output_variable)
signals[upsampled_variable]= signals[output_variable]
current_welle_z= 1000000
for index, row in signals.iterrows():
    if row[upsampled_variable]!= current_welle_z: ### A New value is available
        current_welle_z= row[upsampled_variable]
    else:
        signals.at[index,upsampled_variable]= np.NaN
signals[upsampled_variable].interpolate(method='linear', order=3, axis= 0,inplace= True)
output=  signals[output_variable]


Shift the Z_Welle one step back

In [ ]:
# length= len(signals)
# start= -1
# end= -1
# phases=[]
# for index, row in signals.iterrows():
#     if start<0:
#         start= index
#         continue
#     if row['given2model'] or index == length - 1:
#         end= index-1
#         new_phase= (start, end)
#         phases.append(new_phase)
#         start= index
#         #print(new_phase)
# for phase in phases:
#     start_idx= phase[0]
#     end_idx= phase[1]
#     print(phase)
#     signals['welle_z'][start_idx: end_idx + 1]= signals['welle_z'][end_idx+1]

A function to extract M second array from n second array : M < n

In [ ]:
def extract_time_relevance_array(input_array: np.ndarray, output_shape):
    input_time= input_array.shape[1] # in case of 5 Seconds it will be 240 seconds
    output_time= output_shape[1] # in case of 2 Minutes it will be 10
    step= int(input_time/output_time)
    resut_array= np.zeros(output_shape)
    for i in range( 0, output_time):
        #print('place in result: ', output_time - i - 1, ' place in original: ', input_time - i*step -1)
        resut_array[0, output_time - i - 1, :]= input_array[0, input_time - i*step -1,:]
    return resut_array

Simulate the prediction by passing values with 5 seconds (Input Buffer 2 Minutes, Input data 5 seconds)

In [ ]:
##### Simulate the prediction by passing values with 5 seconds ########
num_model_features= 6
preds= []
input_data= signals.copy()
shape=(1,window,num_model_features)
output_sec_shape= (1, int(window*np.ceil((time_relevance/recording_time))), num_model_features)
print('Requested buffer shape to the model= {}', shape)
print('Orginal buffer shape to the model= {}', output_sec_shape)
first_run= True
inputsignal_list= model_input_Columns
last_prediction_time= None
out= 0
prev_record_time= None
for index, data_point in input_data.iterrows():
    if prev_record_time is not None and (data_point['date']  - prev_record_time).total_seconds() < recording_time:
        print('Escaped!')
        #preds.append(out)
        signals= signals[signals['date']!= data_point['date']]
        continue
    else:
        prev_record_time= data_point['date']
    #print ('index: ', index, ', Date: ', data_point['date'])
    new_signal_list = [float(data_point[sig]) for sig in inputsignal_list]
    #print ('New_signal_list: ', new_signal_list, ' timestamp: ', data_point['date'])
    rough_input_array= np.asarray([new_signal_list])
    signals_ = pipeline.transform(rough_input_array)#np.asarray([new_signal_list])#
    if first_run:
        #print ('index: ', index, ', Date: ', data_point['date'])
        #print('time_ diff= ', str((data_point['date'] - last_prediction_time).total_seconds()))
        # Auffüllen der Struktur - Bei erstem Start ist nur ein Datensatz vorhanden,
        # dieser wird mit np.full in alle kopiert
        five_sec_array= np.float32(np.full(output_sec_shape, signals_[0]))
        array= extract_time_relevance_array(five_sec_array,shape)
        #outputs = ort_sess.run(None, {'x': array})
        outputs =model.predict(array)
        #out = outputs[0][0][0] / 1000
        out = outputs[0][0] / 1000
        #print('Predciction First time. Input Buffer: ', array, 'prediction: ', out)
        preds.append(out)
        first_run = False
        last_prediction_time= data_point['date']
        #print('prediction_time: ', data_point['date'])
        continue
    elif (data_point['date'] - last_prediction_time).total_seconds()< response_time:
            #print('No prediction. Just shift')
            five_sec_array [0][:-1] = five_sec_array[0][1:]
            five_sec_array[0, -1, :] = signals_[0]
            preds.append(out)
            continue 
    else:
        #print ('index: ', index, ', Date: ', data_point['date'])
        #print('time_ diff= ', str((data_point['date'] - last_prediction_time).total_seconds()))
        #print("....Shifting the 5 seconds buffer and insert the new 5s-value")
        five_sec_array [0][:-1] = five_sec_array[0][1:]
        five_sec_array[0, -1, :] = signals_[0]
        array= extract_time_relevance_array(five_sec_array, shape)
        array = np.float32(array)
        ## Apply predicitions on 2 minutes buffer
        outputs =model.predict(array)
        #out = outputs[0][0][0] / 1000
        out = outputs[0][0] / 1000
        #print('Prediction Normal state. Input Buffer: ', array, 'prediction: ', out)
        preds.append(out)
        last_prediction_time= data_point['date']
        #print('prediction_time: ', data_point['date'])
    #signals['local_preds'][index]= out
    #print(signals.iloc[index])
    #print('#########################################################################')

Assign local prediction to the main dataframe

In [ ]:
#signals[output_variable].mask(signals[output_variable]>0.05,0, inplace=True)
signals['local_preds']= preds#[1:]
local_prediction_error= signals[output_variable] - signals['local_preds']
machine_prediction_error= signals[output_variable] - signals[prediction_variable]

In [ ]:
images_path='//Heller.biz/hnt/Steuerungstechnik/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/csvs/smoothed/AI_Model/Backup/Z_Welle/v2.7.0/five_seconds/'
## Comparasion between two sources:
from turtle import color
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['t_bett','t_motor', 't_spindle' ,'given2model','DRZ2', 'M8', 'M121', 'M127', 'M7', prediction_variable]
y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=len(selected_Columns)+1,cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_Columns, vertical_spacing=0.02)
for i in range(len(selected_Columns)):
    fig.add_trace(go.Scatter(x= signals['date'], y= signals[selected_Columns[i]], name=selected_Columns[i], mode= scatter_mode), row= i+1, col= 1)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
##Draw the prediciton and the real values of displacement on Welle
fig.add_trace(go.Scatter(x= signals['date'],y= signals[output_variable], name=output_variable, mode= scatter_mode),  row= len(selected_Columns) , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y=  signals['local_preds'], name='Local predictions', mode= scatter_mode),  row= len(selected_Columns) , col= 1)
fig.update_yaxes(title_text= output_variable, row= len(selected_Columns), col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y=   1000*local_prediction_error, name='New Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y=   1000*machine_prediction_error, name='Machine Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
## Draw the tolerence +-5
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(local_prediction_error,5), name='+5 Max Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(local_prediction_error,-5), name='-5 Min Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)

#fig.add_trace(go.Scatter(x= signals['date'], y=   1000*( np.abs(signals[output_variable] - signals[prediction_variable])), name='Old Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
fig.update_yaxes(title_text= 'Error (Micro-meter)', row= len(selected_Columns)+1, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'Prediction Results')
#pio.write_image(fig, str(images_path +'versuch_25_09_iso.png'), format='png')
fig.show()

Visualize the results

In [ ]:
# # images_path='//Heller.biz/hnt/Steuerungstechnik/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/csvs/smoothed/AI_Model/Backup/Z_Welle/v2.7.0/five_seconds/'
# # ## Comparasion between two sources:
# # from turtle import color
# # scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
# # selected_Columns= [prediction_variable]
# # y_axis_names= selected_Columns
# # #print(preds)
# # fig= make_subplots(rows=len(selected_Columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_Columns, vertical_spacing=0.02)
# # for i in range(len(selected_Columns)):
# #     fig.add_trace(go.Scatter(x= signals['date'], y= signals[selected_Columns[i]], name=selected_Columns[i], mode= scatter_mode), row= i+1, col= 1,)
# #     fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
# # ##Draw the prediciton and the real values of displacement on Welle
# # fig.add_trace(go.Scatter(x= signals['date'],y= signals[output_variable], name=output_variable, mode= scatter_mode),  row= len(selected_Columns) , col= 1)
# # fig.add_trace(go.Scatter(x= signals['date'], y=  signals['local_preds'], name='Local predictions', mode= scatter_mode),  row= len(selected_Columns) , col= 1)
# # fig.update_yaxes(title_text= output_variable, row= len(selected_Columns), col= 1)

# # fig.update_layout(height=1200, width=1400, title_text= 'Prediction Results')
# # #pio.write_image(fig, str(images_path +'versuch_25_09_iso.png'), format='png')
# fig.show()
